In [1]:
import sys
sys.path.insert(0, '../feature_generation')
sys.path.insert(0, '../..')
import pandas as pd
import numpy as np
import keras
import models
import paths
from PIL import Image
import math
from os.path import join
from os import listdir
from keras.preprocessing.image import ImageDataGenerator
from sklearn import model_selection
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
df = pd.read_json(paths.TRAIN_JSON)

In [3]:
listing_id_interest = df[['listing_id', 'interest_level']]

In [6]:
image_interest = []
for listing_id in listing_id_interest.iterrows():
    listing_id_path = join(paths.IMAGES, str(listing_id[1].listing_id))
    for f in listdir(listing_id_path):
        if f.endswith('.jpg'):
            image_interest.append([f,listing_id[1].interest_level])

In [7]:
image_statistics = []
count = 0
for listing_id in listing_id_interest.iterrows():
    listing_id_path = join(paths.IMAGES, str(listing_id[1].listing_id))
    num_images = len(listdir(listing_id_path))
    if num_images == 0:
        continue
    product_sum = 0
    diag_sum = 0
    width_sum = 0
    height_sum = 0
    for f in listdir(listing_id_path):
        if not f.endswith('.jpg'):
            num_images -= 1
            continue
        img = Image.open(join(listing_id_path, f))
        width_sum += img.size[0]
        height_sum += img.size[1]
        product_sum += img.size[0] * img.size[1]
        diag_sum += math.sqrt(img.size[0]**2 + img.size[1]**2)
        count += 1
        if count % 10000 == 0:
            print "finished {} images".format(count)
    if width_sum == 0:
        continue
    image_statistics.append([listing_id[1].listing_id, float(width_sum)/num_images,
                             float(height_sum)/num_images,  
                             float(product_sum)/num_images,
                             float(diag_sum)/num_images])
    
    
        
print len(image_statistics)
npy_file = np.array(image_statistics)
npy_file.shape
np.save('/home/ryan/Desktop/img_stats.npy', npy_file)

In [8]:
# move all the files out of their folders
import os
import shutil

dirList = os.listdir(paths.IMAGES)
dirList = [join(paths.IMAGES,filename) for filename in dirList]
for direc in dirList:
    try:
        fileList = os.listdir(direc)
    except:
        continue
    fileList = [join(direc,filename) for filename in fileList]
    for f in fileList:
        if f.endswith('.jpg'):
            try:
                shutil.move(f,paths.IMAGES)
            except:
                continue
#print fileList[:10]
#for f in fileList:
#    break
    #shutil.move(f, paths.IMAGES)

In [100]:
# remove the empty folders
for i in os.listdir(paths.IMAGES):
    if not i.endswith('.jpg'):
        #print "removing {}".format(join(paths.IMAGES, i))
        try:
            shutil.rmtree(join(paths.IMAGES,i))
        except:
            os.remove(join(paths.IMAGES,i))

In [110]:
# move everything into their labeled folders
for i in image_interest:
#     print "moving {} to {}".format(i[0], i[1])
#     break
    try:
        shutil.move(join(paths.IMAGES,i[0]), join(paths.IMAGES,i[1]))
    except:
        continue

In [114]:
# rest are test images
for i in os.listdir(paths.IMAGES):
    if i.endswith('.jpg'):
        shutil.move(join(paths.IMAGES, i), join(paths.IMAGES,'test'))

In [9]:
# only use one k fold since we're using a cnn
trn_indices = []
val_indices = []
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2016)
for dev_index, val_index in kf.split(range(len(listing_id_interest))):
    trn_indices = dev_index
    val_indices = val_index
    break
print len(trn_indices)
print len(val_indices)

In [160]:
ids = listing_id_interest['listing_id']
ids = np.array(ids)
val_ids = ids[val_indices]

In [168]:
# creating val set
trn_dirs = ['low', 'medium', 'high']
for folder in trn_dirs:
    
    for img in os.listdir(join(paths.IMAGES,folder)):
        listing_id = int(img[:img.index('_')])
        if listing_id in val_ids:
            shutil.move(join(paths.IMAGES,folder,img), 
                                    join(paths.IMAGES,'val',folder,img))